In [1]:
# Here we take care of paths.

from pathlib import Path
import os
print('Starting path:' + os.getcwd())
if os.getcwd()[-18:] == 'VESUVIUS_Challenge':
    pass
else:
    PATH = Path().resolve().parents[0]
    os.chdir(PATH)

# make sure you are in Paragraph_to_Tex folder
print('Current path:' + os.getcwd())

Starting path:/Users/gregory/PROJECT_ML/VESUVIUS_Challenge/jupyter notebooks
Current path:/Users/gregory/PROJECT_ML/VESUVIUS_Challenge


In [2]:
import monai
import torch
import matplotlib.pyplot as plt
import tempfile
import shutil
import os
import glob
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from Data_Modules.MONAI_Dict_Dataset_Module import MONAI_CSV_Scrolls_Dataset
import matplotlib.patches as patches
from lit_models.UNET_monai_lit import UNET_lit
from monai.visualize import matshow3d
from pytorch_lightning.callbacks import ModelCheckpoint

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "mps")

import warnings
warnings.filterwarnings('ignore', category=UserWarning, message='TypedStorage is deprecated')
print(DEVICE)

2023-04-28 14:15:11,448 - Created a temporary directory at /var/folders/wc/60y8v25x3ns_jgsx6clbdb180000gn/T/tmpimyp7hun
2023-04-28 14:15:11,448 - Writing /var/folders/wc/60y8v25x3ns_jgsx6clbdb180000gn/T/tmpimyp7hun/_remote_module_non_scriptable.py
mps


In [3]:
KAGGLE_DIR = PATH / "kaggle"

INPUT_DIR = KAGGLE_DIR / "input"

COMPETITION_DATA_DIR = INPUT_DIR / "vesuvius-challenge-ink-detection"

TRAIN_DATA_CSV_PATH = COMPETITION_DATA_DIR / "data_train_0.5.csv"
#TEST_DATA_CSV_PATH = COMPETITION_DATA_DIR / "data_test_1.0.csv"


In [4]:
dataset = MONAI_CSV_Scrolls_Dataset(
    z_dim=4,
    batch_size=1,
    data_csv_path=TRAIN_DATA_CSV_PATH,
    num_workers=0,
    num_samples=32,
    patch_size=(224,224),
    val_fragment_id=1,
    on_gpu=False,
    
    
)

In [5]:
lit_model = UNET_lit(
        #wandb = True,
        z_dim = 4,
        patch_size = (224,224),
        sw_batch_size=8 ,
        eta_min = 1e-5,
        t_max = 100,
        max_epochs = 1000,
        weight_decay =  3e-4,
        learning_rate = 0.0001,
        gamma = 0.85,)


wandb: Currently logged in as: gmarus. Use `wandb login --relogin` to force relogin


In [6]:
checkpoint_callback = ModelCheckpoint(
    save_top_k=5,
    monitor="val/loss",
    mode="min",
    dirpath="logs/224_smp_unet_32_.5/",
    filename="unet_smp{epoch:02d}{val_loss:.2f}",
)


trainer = pl.Trainer(
        accelerator='gpu',
        #benchmark=True,
        max_epochs=1000,
        check_val_every_n_epoch= 1,
        devices=1,
        #fast_dev_run=fast_dev_run,
        logger=pl.loggers.CSVLogger(save_dir='logs/Test_run/'),
        log_every_n_steps=1,
        default_root_dir = 'logs/Test_run/',
        #overfit_batches=1,
        #precision=16,
        #accumulate_grad_batches=1, 
        #callbacks=[checkpoint_callback],
        #resume_from_checkpoint ='logs/smp_unet_32_.5/lightning_logs/version_3/checkpoints/epoch=610-step=1222.ckpt'
        
        )





trainer.fit(lit_model, datamodule=dataset,
            #ckpt_path='logs/224_smp_unet_32_.5/lightning_logs/version_7/checkpoints/epoch=214-step=430.ckpt'
           )

2023-04-28 14:15:16,915 - GPU available: True (mps), used: True
2023-04-28 14:15:16,916 - TPU available: False, using: 0 TPU cores
2023-04-28 14:15:16,916 - IPU available: False, using: 0 IPUs
2023-04-28 14:15:16,917 - HPU available: False, using: 0 HPUs
# train: 2
# val: 1
Adjusting learning rate of group 0 to 1.0000e-04.
2023-04-28 14:15:17,371 - 
  | Name       | Type                  | Params
-----------------------------------------------------
0 | metrics    | ModuleDict            | 0     
1 | model      | Unet                  | 34.5 M
2 | loss_dice  | DiceLoss              | 0     
3 | loss_bce   | SoftBCEWithLogitsLoss | 0     
4 | loss_focal | FocalLoss             | 0     
-----------------------------------------------------
34.5 M    Trainable params
0         Non-trainable params
34.5 M    Total params
138.108   Total estimated model params size (MB)


/Users/gregory/anaconda3/lib/python3.10/site-packages/lightning_fabric/loggers/csv_logs.py:188: UserWarning: Experiment logs directory logs/Test_run/lightning_logs/version_0 exists and is not empty. Previous log files in this directory will be deleted when the new ones are saved!
  rank_zero_warn(


Sanity Checking: 0it [00:00, ?it/s]

/Users/gregory/anaconda3/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


TypeError: Cannot convert a MPS Tensor to float64 dtype as the MPS framework doesn't support float64. Please use float32 instead.